In [385]:
import re
import math
import sys
sys.path.append("../../src/")
import uncertainpy.gradual as grad
import time
from statistics import median
import random
random.seed(0)

In [386]:
# Parse a QBAF file to an adjacency matrix
def parse_qbaf(filename):
    num_nodes = 0
    with open(filename, 'r') as file:
        for line in file:
            match = re.match(r'(\w+)\((\d+), ([\d.]+)\)', line)
            if match:
                type, node_id1, weight = match.groups()
                if type == 'arg':
                    num_nodes += 1
            else:
                raise ValueError(f"can't parse this line: {line}")
        adj_matrix = [[0] * num_nodes for _ in range(num_nodes)]

    with open(filename, 'r') as file:
        for line in file:
            match = re.match(r'(\w+)\((\d+), ([\d.]+)\)', line)
            if match:
                type, node_id1, node_id2 = match.groups()
                if type == 'att':
                    adj_matrix[int(node_id1)][int(node_id2)] = -1
                elif type == 'sup':
                    adj_matrix[int(node_id1)][int(node_id2)] = 1
            else:
                raise ValueError(f"can't parse this line: {line}")
    return adj_matrix

In [387]:
# Convert the adjacency matrix to a python dictionary
def adjacency_matrix_to_dict(adj_matrix):
    graph_dict = {}

    for i, row in enumerate(adj_matrix):
        neighbors = set(j for j, weight in enumerate(row) if weight != 0)
        graph_dict[i] = neighbors

    return graph_dict

In [388]:
# Compute all the paths between two arguments
def find_all_paths_between_two_args(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    if start not in graph:
        return []
    paths = []
    for node in graph[start]:
        if node not in path:
            new_paths = find_all_paths_between_two_args(graph, node, end, path)
            for new_path in new_paths:
                paths.append(new_path)
    return paths

def find_all_paths_between_two_args_complete(graph, start, end, path=[]):
    if start == end:
        return []
    else:
        paths = find_all_paths_between_two_args(graph, start, end)
        return paths

In [389]:
# Compute the (direct or indirect) polarity from one node to another, return an integer
def compute_polarity_between_two_args(graph, start_node, end_node, adj_matrix):
    paths = find_all_paths_between_two_args_complete(graph, start_node, end_node)
    paths_count = len(paths)
    # print(paths_count)
    # print(paths)

    # case1: self to self:
    if start_node == end_node:
        return 1 # positive

    # case2: no path
    if paths_count == 0:
        return -2 # neutral

    # case3: 1 path or more than 1
    elif paths_count >= 1:

        # record the polarity for each path
        record_polarity = [1] * paths_count
        for j in range(paths_count):
            for i in range(len(paths[j])-1):
                record_polarity[j] *= adj_matrix[paths[j][i]][paths[j][i+1]]

        if all(x == 1 for x in record_polarity):
            pol = 1 # positive: if all the paths are positive
        elif all(x == -1 for x in record_polarity):
            pol = -1 # negative: if all the paths are negative
        else:
            pol = 0 # unknown: some paths positive while some negative

        return pol

In [390]:
# Compute the polarity for all the nodes to the topic argument, return a vector
def compute_polarity_vector(graph, topic_arg, adj_matrix):
    num_nodes = len(graph)
    polarity_vector = [-2] * num_nodes
    for i in range(num_nodes):
        polarity_vector[i] = compute_polarity_between_two_args(graph, i, topic_arg, adj_matrix)
    return polarity_vector

In [391]:
# Compute priority (not polarity) between two arguments
def compute_priority_between_two_args(graph, start, end):
    if start == end:
        return 2 # itself
    paths = find_all_paths_between_two_args(graph, start, end)
    if len(paths) == 0:
        return 0 # disconnected
    min_length = min(len(path) for path in paths) - 1
    priority = 1 / min_length # single/multi-path connected
    return priority

In [392]:
# Compute the priority for all arguments to the topic argument
def compute_priority_vector(graph, topic_arg):
    num_nodes = len(graph)
    priority_vector = [0] * num_nodes
    for i in range(num_nodes):
        priority_vector[i] = compute_priority_between_two_args(graph, i, topic_arg)
    return priority_vector

In [393]:
# Compute different quotiont from one argument to the topic argument
def diff_quotient(bag, arg, topic_arg, h, agg_f, inf_f):

    sigma = bag.arguments[str(topic_arg)].strength

    arg_initial = arg.get_initial_weight()
    arg.reset_initial_weight(arg_initial + h)

    grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
    sigma_new = bag.arguments[str(topic_arg)].strength

    arg.reset_initial_weight(arg_initial)
    grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

    return (sigma_new - sigma) / h

In [394]:
# compute distance: the distance between basescore1 and basescore2
def compute_bs_dist(bs1, bs2):

    l1_dist = 0
    l2_dist = 0

    for item in bs1:
        l1_dist += abs(bs1[item] - bs2[item])
        l2_dist += (bs1[item] - bs2[item]) ** 2

    return l1_dist, math.sqrt(l2_dist)

In [395]:
def compute_potential_cause_acyclic(filename, topic_arg, desired_strength, updating_step, Epoch, polarity, priority, agg_f, inf_f):

    bag = grad.BAG(filename)
    grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
    # print(f"number of args: {len(bag.arguments)}")

    pre_strength = -1
    update_list = [0] * len(bag.arguments)

    # print(f"desired_strength:{desired_strength}")
    initial_strength = bag.arguments[str(topic_arg)].strength
    cur_strength = initial_strength
    print(f"cur_strength:{initial_strength}")

    # compute update_list for disconnected, pos, neg arguments
    for arg in bag.arguments.values():
        if polarity[int(arg.name)] == -2: # disconnected
            update_list[int(arg.name)] = 0
        elif (desired_strength - cur_strength) * polarity[int(arg.name)] > 0:
            # arg.reset_initial_weight(arg.get_initial_weight() + update)
            update_list[int(arg.name)] = updating_step
        elif (desired_strength - cur_strength) * polarity[int(arg.name)] < 0:
            # arg.reset_initial_weight(arg.get_initial_weight() - update)
            update_list[int(arg.name)] = -updating_step

    for epoch in range(Epoch):
        if pre_strength == cur_strength:
            break
        if (desired_strength - cur_strength) * (desired_strength - initial_strength) > 0:
            print(f"Epoch:{epoch}=====================================================")

            # compute update_list for unknown arguments
            for arg in bag.arguments.values():
                if polarity[int(arg.name)] == 0:
                    initial = arg.get_initial_weight()
                    arg.reset_initial_weight(initial + updating_step)
                    grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
                    if abs(desired_strength - bag.arguments[str(topic_arg)].strength) < abs(desired_strength - cur_strength):
                        update_list[int(arg.name)] = updating_step
                    elif abs(desired_strength - bag.arguments[str(topic_arg)].strength) > abs(desired_strength - cur_strength):
                        update_list[int(arg.name)] = -updating_step
                    else:
                        update_list[int(arg.name)] = 0
                    arg.reset_initial_weight(initial)
                    grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

            # finishing computing the update_list
            # print(f"update_list:{update_list}")

            # update all the base scores
            for arg in bag.arguments.values():
                arg.reset_initial_weight(max(0, min(1, arg.get_initial_weight() + update_list[int(arg.get_name())] * priority[int(arg.get_name())])))

            # re-compute the strength
            print(f"desired_strength:{desired_strength}")
            pre_strength = cur_strength
            grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
            cur_strength = bag.arguments[str(topic_arg)].strength
            # print(f"cur_strength:{cur_strength}")

            # print current bs
            current_bs_dict = {}
            for arg in bag.arguments.values():
                current_bs_dict[arg.get_name()] = arg.get_initial_weight()
            # print(f"current base scores: {current_bs_dict}")

    # print current bs
    current_bs_dict = {}
    for arg in bag.arguments.values():
        current_bs_dict[arg.get_name()] = arg.get_initial_weight()
    # print(f"current base scores: {current_bs_dict}")

    # compute validity
    validity = False
    if (desired_strength - cur_strength) * (desired_strength - initial_strength) < 0 or desired_strength == cur_strength:
        validity = True

    return current_bs_dict, validity

In [396]:
def main():

    # Initialising
    N = 100 # number of QBAFs
    runtime_total = 0
    l1_dist_total = 0
    l2_dist_total = 0
    validity_total = 0
    runtime_total_m = []
    l1_dist_total_m = []
    l2_dist_total_m = []

    # Traverse through all the QBAFs
    for i in range(N):
        start_time = time.time() # record the current time
        print(f"N: {i}")

        # Set parameters
        filename = f'../../bags/binary_tree_{i}.bag'
        topic_arg = 0
        desired_strength = 0.5
        updating_step = 0.01
        Epoch = 1000 # iterate 100 times for each QBAF

        # # QE
        # agg_f = grad.semantics.modular.SumAggregation()
        # inf_f = grad.semantics.modular.QuadraticMaximumInfluence(conservativeness=1)

        # # EB
        # agg_f = grad.semantics.modular.SumAggregation()
        # inf_f = grad.semantics.modular.EulerBasedInfluence()

        # DF-QuAD
        agg_f = grad.semantics.modular.ProductAggregation()
        inf_f = grad.semantics.modular.LinearInfluence(conservativeness=1)

        # Obtain origin_base_score_dict
        bag = grad.BAG(filename)
        origin_base_score_dict = {}
        for arg in bag.arguments.values():
            origin_base_score_dict[arg.name] = arg.get_initial_weight()
        # print(f"origin_base_score_dict:{origin_base_score_dict}")


        # Compute polarity, priority
        adj_matrix = parse_qbaf(filename)
        graph_dict = adjacency_matrix_to_dict(adj_matrix)


        polarity = compute_polarity_vector(graph_dict, topic_arg, adj_matrix)
        # priority = compute_priority_vector(graph_dict, topic_arg)
        # polarity = [0] * len(origin_base_score_dict)
        priority = [1] * len(origin_base_score_dict)

        # print(f"cycle polarity:{cycles_polarity_dict}")
        # print(f"polarity:{polarity}")
        # print(f"priority:{priority}")


        # Compute a potential cause
        potential_cause_dict, validity = compute_potential_cause_acyclic(filename, topic_arg, desired_strength, updating_step, Epoch, polarity, priority, agg_f, inf_f)

        # Compute validity
        if validity:
            validity_total += 1

        # Compute distance
        l1_dist, l2_dist = compute_bs_dist(origin_base_score_dict, potential_cause_dict)

        l1_dist_total += l1_dist
        l2_dist_total += l2_dist

        print(f"L1_dist:{l1_dist}")
        print(f"L2_dist:{l2_dist}")

        end_time = time.time()
        runtime = end_time - start_time
        print(f"Runtime: {runtime}")
        runtime_total += runtime

        l1_dist_total_m.append(l1_dist)
        l2_dist_total_m.append(l2_dist)
        runtime_total_m.append(runtime)

    print(f"================================================")
    print(f"Summary Results for {N} QBAFs:")
    # print(f"Validity_avg: {validity_total}%")
    # print(f"L1_dist_avg: {l1_dist_total/N}")
    # print(f"L2_dist_avg: {l2_dist_total/N}")
    # print(f"Runtime_avg: {runtime_total/N}")
    print(f"{validity_total}%")
    print(f"{l1_dist_total/N}")
    print(f"{l2_dist_total/N}")
    print(f"{runtime_total/N}")
    # print(f"L1_median: {median(l1_dist_total_m)}")
    # print(f"L2_median: {median(l2_dist_total_m)}")
    # print(f"Runtime_median: {median(runtime_total_m)}")

if __name__ == "__main__":
    main()

N: 0
cur_strength:0.9741470901934904
Epoch:0=====================================================
desired_strength:0.5
Epoch:1=====================================================
desired_strength:0.5
Epoch:2=====================================================
desired_strength:0.5
Epoch:3=====================================================
desired_strength:0.5
Epoch:4=====================================================
desired_strength:0.5
Epoch:5=====================================================
desired_strength:0.5
Epoch:6=====================================================
desired_strength:0.5
Epoch:7=====================================================
desired_strength:0.5
Epoch:8=====================================================
desired_strength:0.5
Epoch:9=====================================================
desired_strength:0.5
Epoch:10=====================================================
desired_strength:0.5
Epoch:11====================================================